# Loading libaries

In [1]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from scipy import stats
import matplotlib.pyplot as plt
from copy import deepcopy
from plotly.subplots import make_subplots

# Freq change

## Loading Data

In [2]:
DIR = 'Results/Catch22/'

In [3]:
df_real_1 = pd.read_csv(DIR+'features_df_signal_1_real_data_frame_sine_freq_change_double_22.csv', index_col=0).add_prefix('real_1_')
df_real_2 = pd.read_csv(DIR+'features_df_signal_2_real_data_frame_sine_freq_change_double_22.csv', index_col=0).add_prefix('real_2_')

df_generated_without_CD_1 = pd.read_csv(DIR+'Without_CD/features_df_signal_1_generated_data_frame_sine_freq_change_double_22.csv', index_col=0).add_prefix('generated_without_CD_1_')
df_generated_without_CD_2 = pd.read_csv(DIR+'Without_CD/features_df_signal_2_generated_data_frame_sine_freq_change_double_22.csv', index_col=0).add_prefix('generated_without_CD_2_')

df_generated_with_CD_1 = pd.read_csv(DIR+'With_CD/features_df_signal_1_generated_data_frame_sine_freq_change_double_22.csv', index_col=0).add_prefix('generated_with_CD_1_')
df_generated_with_CD_2 = pd.read_csv(DIR+'With_CD/features_df_signal_2_generated_data_frame_sine_freq_change_double_22.csv', index_col=0).add_prefix('generated_with_CD_2_')

df_real_1

,real_1_DN_HistogramMode_5,real_1_DN_HistogramMode_10,real_1_CO_f1ecac,real_1_CO_FirstMin_ac,real_1_CO_HistogramAMI_even_2_5,real_1_CO_trev_1_num,real_1_MD_hrv_classic_pnn40,real_1_SB_BinaryStats_mean_longstretch1,real_1_SB_TransitionMatrix_3ac_sumdiagcov,real_1_PD_PeriodicityWang_th0_01,...,real_1_DN_OutlierInclude_p_001_mdrmd,real_1_DN_OutlierInclude_n_001_mdrmd,real_1_SP_Summaries_welch_rect_area_5_1,real_1_SB_BinaryStats_diff_longstretch0,real_1_SB_MotifThree_quantile_hh,real_1_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,real_1_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,real_1_SP_Summaries_welch_rect_centroid,real_1_FC_LocalSimple_mean3_stderr,real_1_ID
0,0.032502,-0.206385,41.0,2.0,0.818370,-0.004426,0.902378,95.0,0.005102,142.0,...,0.058125,0.043750,0.954503,7.0,1.909906,0.872340,0.702128,0.03068,0.274174,2.0
1,-0.051036,0.198465,41.0,2.0,0.762021,-0.005457,0.901126,99.0,0.030612,147.0,...,0.008750,-0.067500,0.950855,7.0,1.897223,0.872340,0.702128,0.03068,0.291078,1.0
2,-0.080784,0.155335,41.0,95.0,0.869863,-0.001336,0.884856,97.0,0.010204,147.0,...,-0.760000,0.012500,0.971122,6.0,1.887535,0.851064,0.680851,0.03068,0.222472,2.0
3,0.047439,-0.202974,41.0,97.0,0.889466,0.000048,0.893617,98.0,0.005102,151.0,...,-0.131250,0.132500,0.964535,5.0,1.915206,0.872340,0.702128,0.03068,0.246474,2.0
4,0.007462,-0.245670,40.0,2.0,0.626327,0.005400,0.926158,95.0,0.003945,252.0,...,-0.102500,0.240000,0.924104,5.0,1.955431,0.148936,0.702128,0.03068,0.357590,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.083379,-0.182761,39.0,4.0,0.391532,0.024415,0.934919,79.0,0.021696,157.0,...,0.066250,0.155000,0.840543,6.0,1.996181,0.255319,0.617021,0.03068,0.516753,1.0
996,0.019029,-0.220415,41.0,1.0,0.731299,0.005637,0.924906,96.0,0.003945,147.0,...,0.086250,-0.020625,0.946827,6.0,1.904105,0.872340,0.680851,0.03068,0.300222,1.0
997,0.040461,-0.226942,40.0,4.0,0.581497,-0.010563,0.909887,90.0,0.020408,142.0,...,-0.485000,-0.010000,0.902821,5.0,1.921212,0.148936,0.617021,0.03068,0.404061,1.0
998,0.042936,-0.235348,39.0,1.0,0.544322,-0.010167,0.949937,90.0,0.025510,139.0,...,-0.790000,0.062500,0.899584,6.0,1.956665,0.170213,0.638298,0.03068,0.410621,1.0


## Correlation Analysis

### Computing Correlation

In [4]:
real_corr = pd.concat([df_real_1.drop(columns=["real_1_ID"]), df_real_2.drop(columns=["real_2_ID"])], axis=1).corr()
real_corr = real_corr[list(df_real_2.columns)[:-1]].loc[list(df_real_1.columns)[:-1]]

generated_without_CD = pd.concat([df_generated_without_CD_1, df_generated_without_CD_2], axis=1).corr()
generated_without_CD = generated_without_CD[list(df_generated_without_CD_2.columns)].loc[list(df_generated_without_CD_1.columns)]

generated_with_CD = pd.concat([df_generated_with_CD_1, df_generated_with_CD_2], axis=1).corr()
generated_with_CD = generated_with_CD[list(df_generated_with_CD_2.columns)].loc[list(df_generated_with_CD_1.columns)]

indices_real = real_corr.columns[real_corr.isna().all()].tolist()
indices_generated_without_CD = generated_without_CD.columns[generated_without_CD.isna().all()].tolist()
indices_generated_with_CD = generated_with_CD.columns[generated_with_CD.isna().all()].tolist()

indices_total = indices_real + indices_generated_without_CD + indices_generated_with_CD

features = ['SP_Summaries_welch_rect_centroid', 'FC_LocalSimple_mean1_tauresrat']
real_features_2 = ['real_2_'+f for f in features]
generated_without_CD_features_2 = ['generated_without_CD_2_'+f for f in features]
generated_with_CD_features_2 = ['generated_with_CD_2_'+f for f in features]

real_features_1 = ['real_1_'+f for f in features]
generated_without_CD_features_1 = ['generated_without_CD_1_'+f for f in features]
generated_with_CD_features_1 = ['generated_with_CD_1_'+f for f in features]

real_not_nan = real_corr.drop(real_features_2, axis=1)
generated_with_CD_not_nan = generated_with_CD.drop(generated_with_CD_features_2, axis=1)
generated_without_CD_not_nan = generated_without_CD.drop(generated_without_CD_features_2, axis=1)

real_not_nan = real_not_nan.drop(real_features_1, axis=0)
generated_with_CD_not_nan = generated_with_CD_not_nan.drop(generated_with_CD_features_1, axis=0)
generated_without_CD_not_nan = generated_without_CD_not_nan.drop(generated_without_CD_features_1, axis=0)

In [5]:
generated_without_CD_not_nan

,generated_without_CD_2_DN_HistogramMode_5,generated_without_CD_2_DN_HistogramMode_10,generated_without_CD_2_CO_f1ecac,generated_without_CD_2_CO_FirstMin_ac,generated_without_CD_2_CO_HistogramAMI_even_2_5,generated_without_CD_2_CO_trev_1_num,generated_without_CD_2_MD_hrv_classic_pnn40,generated_without_CD_2_SB_BinaryStats_mean_longstretch1,generated_without_CD_2_SB_TransitionMatrix_3ac_sumdiagcov,generated_without_CD_2_PD_PeriodicityWang_th0_01,generated_without_CD_2_CO_Embed2_Dist_tau_d_expfit_meandiff,generated_without_CD_2_IN_AutoMutualInfoStats_40_gaussian_fmmi,generated_without_CD_2_DN_OutlierInclude_p_001_mdrmd,generated_without_CD_2_DN_OutlierInclude_n_001_mdrmd,generated_without_CD_2_SP_Summaries_welch_rect_area_5_1,generated_without_CD_2_SB_BinaryStats_diff_longstretch0,generated_without_CD_2_SB_MotifThree_quantile_hh,generated_without_CD_2_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,generated_without_CD_2_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,generated_without_CD_2_FC_LocalSimple_mean3_stderr
generated_without_CD_1_DN_HistogramMode_5,0.045416,0.003538,-0.009393,-0.003619,-0.011249,0.112248,-0.007781,-0.058062,-0.040523,0.033746,0.000275,0.013965,0.075730,0.005323,0.000899,0.024371,0.019890,-0.006794,0.017750,-0.000745
generated_without_CD_1_DN_HistogramMode_10,-0.041384,-0.015234,0.041692,-0.024086,0.036582,-0.048362,-0.043839,0.015930,-0.042224,0.004721,0.028606,-0.009927,0.029023,-0.021149,0.033724,0.024067,-0.034062,0.050755,0.008195,-0.035071
generated_without_CD_1_CO_f1ecac,-0.008227,-0.014884,0.028376,0.049145,0.023865,-0.028334,0.001528,-0.012622,-0.060474,0.034062,0.045149,0.018604,0.093617,-0.047422,0.034831,-0.046452,0.009311,0.019573,-0.011979,-0.030346
generated_without_CD_1_CO_FirstMin_ac,-0.092916,-0.014880,0.046467,0.008177,0.044891,-0.055267,-0.014386,0.028129,0.015642,0.049340,0.040069,0.030753,0.079750,-0.063213,0.048280,0.003030,-0.020983,-0.004863,-0.002134,-0.042203
generated_without_CD_1_CO_HistogramAMI_even_2_5,-0.040982,-0.013009,0.049231,0.031613,0.028773,-0.044410,-0.006187,-0.017996,-0.043649,0.036230,0.051587,-0.003515,0.148343,-0.041405,0.039067,-0.007580,-0.004717,0.026892,0.006073,-0.034036
generated_without_CD_1_CO_trev_1_num,0.032905,-0.004724,-0.020154,0.032822,-0.041328,0.032459,0.029035,-0.037095,0.052939,-0.018874,-0.062590,-0.007271,-0.084514,-0.025979,-0.038179,-0.027114,0.007270,-0.056749,0.016124,0.041371
generated_without_CD_1_MD_hrv_classic_pnn40,0.038246,0.000302,-0.052177,-0.002301,-0.029258,0.057875,0.000286,0.047156,0.005934,-0.029370,-0.055597,-0.001026,-0.107456,0.046170,-0.035078,0.002267,0.011505,-0.013265,0.007326,0.030936
generated_without_CD_1_SB_BinaryStats_mean_longstretch1,0.033847,-0.024075,-0.016263,0.027794,0.017886,0.042372,-0.022001,-0.006609,-0.110404,0.004865,0.047680,-0.021133,0.108804,-0.046500,0.028094,0.024254,-0.001411,0.011879,0.016947,-0.027605
generated_without_CD_1_SB_TransitionMatrix_3ac_sumdiagcov,-0.026362,-0.028912,0.010371,0.001884,0.028774,0.000190,-0.018700,-0.020991,0.035666,0.020266,0.056716,0.010428,-0.022001,-0.007674,0.021571,0.008387,-0.030400,-0.034633,-0.035295,-0.023422
generated_without_CD_1_PD_PeriodicityWang_th0_01,-0.025332,0.017297,-0.027654,0.016745,0.014414,0.005867,-0.000064,-0.013524,-0.083418,0.016483,0.036074,0.006732,0.103125,-0.019039,0.017494,0.004569,-0.008118,0.013533,-0.005727,-0.019859


### Plotting Correlation

In [6]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Real Data',
                                    'Without CD', 'With CD'))

fig.add_trace(
    px.imshow(real_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=1
)
# fig.update_layout(
#     yaxis_title = 'Channel 2',
#     xaxis_title = 'Channel 1',
#     )
fig.add_trace(
    px.imshow(generated_without_CD_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=2
)

fig.add_trace(
    px.imshow(generated_with_CD_not_nan, 
                labels=dict(x="Channel 1", y="Channel 2", color="Correlation"),
                color_continuous_scale=px.colors.diverging.RdBu, 
                range_color=(-1, 1)).data[0], row=1, col=3
)
fig.update_xaxes(showticklabels=False) 
fig.update_yaxes(showticklabels=False) 
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 1)
fig.update_yaxes(title_text="Feature Channel 2", row = 1, col = 1)
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 2)
fig.update_xaxes(title_text="Feature Channel 1", row = 1, col = 3)

fig.update_layout(height=240, width=800, margin = {'l':0,'r':0,'t':23,'b':0},)
fig.show()

In [12]:
fig.write_image("Sine_freq_change_correlation.pdf")

### Spearman's rho

In [7]:
SPR_without_CD_1 = stats.spearmanr(np.array(real_not_nan).flatten(), np.array(generated_without_CD_not_nan).flatten())
SPR_with_CD_1 = stats.spearmanr(np.array(real_not_nan).flatten(), np.array(generated_with_CD_not_nan).flatten())

print("SPR for Signal 1 without CD: ", SPR_without_CD_1)
print("SPR for Signal 1 with CD: ", SPR_with_CD_1)

SPR for Signal 1 without CD:  SpearmanrResult(correlation=0.25888380552378454, pvalue=1.5106077120169357e-07)
SPR for Signal 1 with CD:  SpearmanrResult(correlation=0.8337520859505372, pvalue=1.0362922342058581e-104)


### Kendall's Tau

In [8]:
KT_without_CD_1 = stats.kendalltau(np.array(real_not_nan).flatten(), np.array(generated_without_CD_not_nan).flatten())
KT_with_CD_1 = stats.kendalltau(np.array(real_not_nan).flatten(), np.array(generated_with_CD_not_nan).flatten())

print("KT for Signal 1 without CD: : ", KT_without_CD_1)
print("KT for Signal 1 with CD: : ", KT_with_CD_1)

KT for Signal 1 without CD: :  KendalltauResult(correlation=0.17398496240601508, pvalue=2.0274214897526866e-07)
KT for Signal 1 with CD: :  KendalltauResult(correlation=0.6612030075187971, pvalue=8.076280946256558e-87)


### Frobenius Norm

In [9]:
FBNorm_without_CD_1 = np.linalg.norm(np.array(real_not_nan) - np.array(generated_without_CD_not_nan))
FBNorm_with_CD_1 = np.linalg.norm(np.array(real_not_nan )- np.array(generated_with_CD_not_nan))

print("Frobenius Signal for Real 1 without CD: ", FBNorm_without_CD_1)
print("Frobenius Signal for Real 1 with CD: ", FBNorm_with_CD_1)

Frobenius Signal for Real 1 without CD:  7.889015896350542
Frobenius Signal for Real 1 with CD:  3.412918358342001


### MAE

In [10]:
MSE_without_CD_1 = np.mean(np.abs(np.array(real_not_nan) - np.array(generated_without_CD_not_nan)))
MSE_with_CD_1 = np.mean(np.abs(np.array(real_not_nan) - np.array(generated_with_CD_not_nan)))

print("MSE of Real Data 1 and Generated Data without CD 1: ", MSE_without_CD_1)
print("MSE of Real Data 1 and Generated Data with CD 1: ", MSE_with_CD_1)

MSE of Real Data 1 and Generated Data without CD 1:  0.26836166880406637
MSE of Real Data 1 and Generated Data with CD 1:  0.13147473752783262
